In [ ]:
%matplotlib inline
import sys
sys.path.append('../')
sys.path.append('../../')
sys.path.append('../../../')
sys.path.append('../../../../')
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime, timedelta
from matplotlib import pyplot as plt

from PyFin.api import *
from alphamind.api import *
from conf.models import *
from conf.config import*
from data.engines.model import Record2, Record
from alphamind.execution.naiveexecutor import NaiveExecutor
from stacking import factor_store, feature_list
from optimization.bayes_optimization_xgb import *
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',100)

from data.engines.sqlengine import SQLEngine

weekly = 's2'  # s1, s2,  s3, s4, s5, s_1
record_engine = SQLEngine('sqlite:///./{}/real_tune_record_without_alpha.db'.format(weekly))


In [ ]:
# ref_date_pre = '2019-12-27'

previous_pos = record_engine.fetch_record_meta(Record2, trade_date=None)
previous_pos

In [ ]:
previous_pos['adjust_date'] = previous_pos.trade_date
previous_pos['trade_date'] = previous_pos.trade_date + timedelta(days=1)
previous_pos

In [ ]:
record_engine.write_data('pos_record', previous_pos, if_exists='replace')

In [ ]:
# record_engine.fetch_record('pos_record')
previous_pos = record_engine.fetch_record_meta(Record, trade_date=None)


In [ ]:
previous_pos